In [363]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

In [364]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [365]:
covid_df = pd.read_excel('แบบสอบถามเรื่องผลกระทบจาก COVID-19 กับนักพัฒนาซอฟต์แวร์_shortened (Responses)-2nd_round.xlsx')
original_covid_df = covid_df.copy()

In [366]:
# covid_df.head()


In [367]:
covid_df.shape

(167, 163)

In [368]:
covid_df.columns.tolist()

['Timestamp',
 'โปรดยืนยันว่าท่านได้อ่านและเข้าใจเนื้อหาในเอกสารชี้แจงผู้เข้าร่วมวิจัย โดยการตอบแบบสอบถาม (Self-Administered Questionnaire Participant Information Sheet) แล้ว',
 'ท่านเคยทำงานในที่ทำงาน (ออฟฟิศ) อย่างน้อยเป็นเวลา 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน',
 'A1 ตำแหน่งงานปัจจุบันของท่าน',
 'A2 ท่านมีประสบการณ์ในการทำงานด้านพัฒนาซอฟต์แวร์มาแล้วเป็นเวลากี่ปี? ',
 'A3 ท่านมีประสบการณ์ในการทำงานที่บ้านมาแล้วเป็นเวลากี่ปี?',
 'A4 ตำแหน่งงานหลักของข้าพเจ้าเป็นแบบ',
 'A5 ท่านมีคนอาศัยอยู่ในบ้านเดียวกันกับท่านจำนวนกี่คน (รวมตัวท่านเอง)?',
 'A6 คนที่อาศัยอยู่ในบ้านเดียวกับท่าน มีกี่คนที่อายุน้อยกว่า 12 ปี?',
 'A7 ท่านอายุเท่าไหร่?',
 'A8 โปรดระบุเพศของท่าน',
 'A9 ท่านจบการศึกษาสูงสุดในระดับใด?',
 'A10 บริษัทของท่าน (ไม่ใช่ทีมของท่าน) มีจำนวนพนักงานประมาณกี่คน?',
 'A11 ท่านมีความพิการทางร่างกายที่มีผลต่อการทำงานของท่านหรือไม่?',
 'B1 โปรดเลือกข้อที่เหมาะสมกับคุณที่สุด',
 'C1 โปรดเลือกทุกข้อที่ตรงกับสถานะของท่าน',
 'C2 เลือกข้อที่ตรงกับท่านมากที่สุด',
 'D1 ท่า

In [369]:
covid_df.describe()

,I1 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของคนส่วนใหญ่ที่ทำงานคล้ายๆ กับท่านเท่าไร?,I2 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วง 1 หรือ 2 ปีที่ผ่านมาเท่าไร?,I3 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วงที่ท่านเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 เท่าไร?
count,146.000000,145.000000,146.000000
mean,7.301370,7.751724,7.698630
std,1.371482,1.402037,1.599008
min,1.000000,1.000000,2.000000
25%,6.250000,8.000000,7.000000
50%,8.000000,8.000000,8.000000
75%,8.000000,8.000000,9.000000
max,10.000000,10.000000,10.000000


In [370]:

covid_df = covid_df.rename(columns={'โปรดยืนยันว่าท่านได้อ่านและเข้าใจเนื้อหาในเอกสารชี้แจงผู้เข้าร่วมวิจัย โดยการตอบแบบสอบถาม (Self-Administered Questionnaire Participant Information Sheet) แล้ว':'Confirm_REMOVE',
'ท่านเคยทำงานในที่ทำงาน (ออฟฟิศ) อย่างน้อยเป็นเวลา 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน':'Office_work_before_REMOVE',
'A1 ตำแหน่งงานปัจจุบันของท่าน':'RolesIncludeDeveloper',
'A2 ท่านมีประสบการณ์ในการทำงานด้านพัฒนาซอฟต์แวร์มาแล้วเป็นเวลากี่ปี? ':'YearsOfExperience',
'A3 ท่านมีประสบการณ์ในการทำงานที่บ้านมาแล้วเป็นเวลากี่ปี?':'YearsOfWorkFromHomeExperience',
'A4 ตำแหน่งงานหลักของข้าพเจ้าเป็นแบบ':'Fulltime',
'A5 ท่านมีคนอาศัยอยู่ในบ้านเดียวกันกับท่านจำนวนกี่คน (รวมตัวท่านเอง)?':'Coinhabitants',
'A6 คนที่อาศัยอยู่ในบ้านเดียวกับท่าน มีกี่คนที่อายุน้อยกว่า 12 ปี?':'Children',
'A7 ท่านอายุเท่าไหร่?':'Age',
'A8 โปรดระบุเพศของท่าน':'Gender',
'A9 ท่านจบการศึกษาสูงสุดในระดับใด?':'Education',
'A10 บริษัทของท่าน (ไม่ใช่ทีมของท่าน) มีจำนวนพนักงานประมาณกี่คน?':'OrganizationSize',
'A11 ท่านมีความพิการทางร่างกายที่มีผลต่อการทำงานของท่านหรือไม่?':'Disabilities',
'B1 โปรดเลือกข้อที่เหมาะสมกับคุณที่สุด':'B1_REMOVE',
'C1 โปรดเลือกทุกข้อที่ตรงกับสถานะของท่าน':'COVIDStatus0-5',
'C2 เลือกข้อที่ตรงกับท่านมากที่สุด':'Isolation',
'D1 ท่านรู้สึกกลัวว่า [ท่านจะติดเชื้อ COVID-19]':'FR1',
'D1 ท่านรู้สึกกลัวว่า [ท่านจะเสียชีวิตจากการติดเชื้อ COVID-19]':'FR2',
'D1 ท่านรู้สึกกลัวว่า [คนในครอบครัวของท่านจะเสียชีวิตจากการติดเชื้อ COVID-19]':'FR3',
'D2 ตอนนี้ท่านรู้สึก [หวาดกลัว]':'FR4',
'D2 ตอนนี้ท่านรู้สึก [ตนเองไม่สามารถทำอะไรเพื่อแก้ปัญหาได้]':'FR5',
'D2 ตอนนี้ท่านรู้สึก [ตกใจกลัวอย่างที่สุด]':'FR6',
'D3 โปรดตามคำถามต่อไปนี้ [ท่านหวาดกลัวว่าท่านจะไม่มีเงินหากท่านไม่สามารถทำงานได้ในช่วง 2-3 เดือนข้างหน้าหรือไม่?]':'FR7',
'D3 โปรดตามคำถามต่อไปนี้ [ปกติแล้วท่านสามารถหายจากอาการเจ็บป่วยได้รวดเร็วหรือไม่?]':'FR8',
'D3 โปรดตามคำถามต่อไปนี้ [ท่านมีเพื่อนที่สามารถติดต่อได้ทางโทรศัพท์ อีเมล หรืออื่นๆ หรือไม่ ในเวลาที่ท่านรู้สึกโดดเดี่ยว]':'FR9',
'How many different prescription medications are you on?':'FR10',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าปรึกษากับคนอื่นเกี่ยวกับข้อมูลเรื่อง COVID-19 ที่ข้าพเจ้าได้รับมา]':'DP1',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้ามีชุดยังชีพฉุกเฉิน]':'DP2',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าได้วางแผนไว้แล้วว่าต้องปฏิบัติตัวอย่างไรหากข้าพเจ้าตรวจพบเชื้อ COVID-19]':'DP3',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าปฏิบัติตามคำแนะนำของหน่วยงานของรัฐเกี่ยวกับการปฏิบัติตัวช่วง COVID-19 อย่างเคร่งครัด]':'DP4',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้ารู้จักคนที่จะช่วยข้าพเจ้าได้หากข้าพเจ้าจะต้องกักตัว]':'DP5',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [สถาพแวดล้อมของที่ทำงานที่บ้านของข้าพเจ้าปราศจากสิ่งรบกวน]':'Erg1',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [ระดับของเสียงรบกวนที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่ยอมรับได้ (ไม่รบกวนการทำงาน)]':'Erg2',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [อุณหภูมิของที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่เหมาะสม]':'Erg3',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [แสงในที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่เหมาะสม]':'Erg4',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [เก้าอี้ทำงานของข้าพเจ้ามีความสบาย]':'Erg5',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [ที่ทำงานที่บ้านของข้าพเจ้ามีความเหมาะสมกับการทำงาน]':'Erg6',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ข้าพเจ้าสามารถนำอุปกรณ์ (เช่น จอภาพ) จากที่ทำงานกลับมาที่บ้านได้]':'OS1',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะซื้ออุปกรณ์ใหม่ให้พนักงานเพื่อทำงานที่บ้าน]':'OS2',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะซื้อซอฟต์แวร์ที่จำเป็นสำหรับการทำงานที่บ้าน]':'OS3',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะจ่ายค่าอินเตอร์เน็ตให้ข้าพเจ้า]':'OS4',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าจะไม่ถูกไล่ออก]':'OS5',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าจะได้รับเงินเดือนเหมือนเดิม]':'OS6',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าสามารถลาหยุดหากข้าพเจ้าป่วยหรือลาเพื่อไปดูแลคนในครอบครัวได้]':'OS7',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าองค์กรมีความเข้าใจหากผลการทำงานของข้าพเจ้าแย่ลง]':'OS8',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ายังคงมีการประชุมประจำเช่นเดิม (เช่น ผ่านระบบ video call)]':'OS9',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการประชุม (เช่น ผ่าน video call) น้อยลง]':'OS10',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าให้กำลังใจพนักงาน โดยให้มาพบเจอกัน (แบบออนไลน์) เป็นประจำ]':'OS11',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการสร้างกิจกรรม virtual social events (ผ่านระบบ via video chat) ให้ทุกคนในทีม]':'OS12',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ในแต่ละวัน ข้าพเจ้าทำงานโดยเปิด video call หรือ audio call ค้างไว้กับเพื่อนร่วมงานในทีม]':'OS13',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าส่งอาหารให้กับพนักงานที่บ้าน]':'OS14',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ามีกิจกรรมให้บุตรของข้าพเจ้าได้ทำระหว่างที่ข้าพเจ้าทำงาน]':'OS15',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ามีกิจกรรมให้พนักงานออกกำลังกายที่บ้าน]':'OS16',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าสนับสนุนให้พนักงานใช้เวลาช่วงนี้ในการฝึกฝนตนเอง (professional training)]':'OS17',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีระบบที่รองรับการทำงานที่บ้านได้อย่างดี (เช่น source control, VPN, remote desktop, file sharing)]':'OS18',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการตรวจเช็ค commits, change request หรือ pull request (peer code review)]':'OS19',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้าใช้ระบบ build system เพื่อทำการ compile และ test software]':'OS20',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ากำลังปรับปรุงเอกสารและกระบวนการทำงาน (เช่น การ approve change request/pull request)]':'OS21',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [มีคนในองค์กรที่ช่วยจัดลำดับความสำคัญของงานและตรวจเช็คข้อมูลใน backlog ให้ถูกต้องเสมอ]':'OS22',
'F2 มีสิ่งอื่นใดที่บริษัทของท่านกำลังทำหรือควรจะทำเพื่อช่วยท่านระหว่างทำงานที่บ้าน?':'Temp1',
'F3 มีสิ่งใดที่บริษัทของท่านทำแล้วท่านคิดว่าทำให้การทำงานที่บ้านลำบากมากขึ้น?':'Temp2',

'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกสดชื่นและมีกำลังใจดี]':'WHO5B1',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกสงบและผ่อนคลาย]':'WHO5B2',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกตื่นตัวและกระฉับกระเฉง]':'WHO5B3',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้าตื่นมาแล้วรู้สึกสดชื่นและพักผ่อนอย่างเพียงพอ]':'WHO5B4',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [กิจกรรมในแต่ละวันของข้าพเจ้าเต็มไปด้วยส่ิงที่น่าสนใจ]':'WHO5B5',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ประสิทธิภาพการทำงานของท่านสูงกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQB1',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ประสิทธิภาพการทำงานของท่านต่ำกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQB2',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านเลือกที่จะไม่ทำงานทั้งที่ควรจะต้องทำงานนั้นๆ บ่อยแค่ไหน?]':'HPQB3',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านทำงานไม่ละเอียดเท่าที่ควรบ่อยแค่ไหน?]':'HPQB4',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านทำงานได้คุณภาพต่ำกว่าที่ควรจะเป็นบ่อยแค่ไหน?]':'HPQB5',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านไม่มีสมาธิกับการทำงานบ่อยแค่ไหน?]':'HPQB6',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [บ่อยแค่ไหนที่ความกังวลเกี่ยวกับสถานการณ์ของ COVID-19 ทำให้ท่านทำงานได้น้อยลง?]':'HPQB7',
'G3 หากท่านเปรียบเทียบประสิทธิภาพในการทำงานของท่านกับคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน ในช่วง 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน ท่านคิดว่าอย่างไร?':'HPQB8',
'G4 ตอนที่ท่านทำงานที่ออฟฟิศ ในบริเวณโต๊ะที่ท่านทำงานมีคนจำนวนกี่คน (รวมตัวท่านเอง)?':'Temp3',
'G5 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านมีประชุมวันละประมาณกี่ครั้ง?':'Temp4',
'G5 ตอนที่ท่านทำงานที่ออฟฟิศ ทีมของท่านมีการประชุมแบบ stand-up meeting เป็นประจำหรือไม่?':'Temp5',
'G6 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านทำงานร่วมกับเพื่อนร่วมงานอีกประมาณกี่คน?':'Temp6',
'G7 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านใช้เวลาเดินทาง (นาที) โดยเฉลี่ยเท่าไหร่ต่อวัน โปรดรวมระยะเวลาทั้งขาไปและขากลับ':'Temp7',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกสดชื่นและมีกำลังใจดี]':'WHO5S1',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกสงบและผ่อนคลาย]':'WHO5S2',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกตื่นตัวและกระฉับกระเฉง]':'WHO5S3',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้าตื่นมาแล้วรู้สึกสดชื่นและพักผ่อนอย่างเพียงพอ]':'WHO5S4',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [กิจกรรมในแต่ละวันของข้าพเจ้าเต็มไปด้วยส่ิงที่น่าสนจ]':'WHO5S5',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ประสิทธิภาพการทำงานของท่านสูงกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQS1',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ประสิทธิภาพการทำงานของท่านต่ำกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQS2',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านไม่ทำงานทั้งที่ควรจะต้องทำงานบ่อยแค่ไหน?]':'HPQS3',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านทำงานไม่ละเอียดเท่าที่ควรบ่อยแค่ไหน?]':'HPQS4',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านทำงานได้คุณภาพต่ำกว่าที่ควรจะเป็นบ่อยแค่ไหน?]':'HPQS5',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านไม่มีสมาธิกับการทำงานบ่อยแค่ไหน?]':'HPQS6',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [บ่อยแค่ไหนที่ความกังวลเกี่ยวกับสถานการณ์ของ COVID-19 ทำให้ท่านทำงานได้น้อยลง?]':'HPQS7',
'H3 หากท่านเปรียบเทียบประสิทธิภาพในการทำงานของท่านกับคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน ในช่วง 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ตั้งแต่ท่านจะเริ่มทำงานที่บ้าน ท่านคิดว่าอย่างไร?':'HPQS8',
'H4 ตอนที่ท่านทำงานที่บ้าน ทีมของท่านมีการประชุมแบบ stand-up meeting เป็นประจำหรือไม่?':'Temp7.1',
'H5 ท่านทำงานที่บ้านเป็นประจำหรือไม่ก่อนเกิดการแพร่ระบาดของ COVID-19?':'Temp8',
'H5 ก่อนที่ท่านจะเริ่มทำงานที่บ้าน บุตรของท่านไปเรียนที่โรงเรียนหรือเนิร์สเซอรี่หรือไม่':'Temp9',
'H6 การที่ต้องดูแลบุตรของท่านระหว่างการแพร่ระบาดของ COVID-19 เป็นเรื่องง่ายหรือยากแค่ไหนสำหรับท่าน?':'Temp10',
#'H8 โปรดอธิบายว่าสถานที่ทำงานที่บ้านของท่านเป็นอย่างไร ':'Temp11',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Laptop]':'Temp12',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Desktop]':'Temp13',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Multiple screens]':'Temp14',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Headphones]':'Temp15',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Keyboard]':'Temp16',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Mouse]':'Temp17',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [โต๊ะ]':'Temp18',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Whiteboard]':'Temp19',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [โต๊ะแบบ Ergonomic]':'Temp20',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [เก้าอี้แบบ Ergonomic]':'Temp21',
#'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [แสงสว่างตามธรรมชาติ]':'Temp22',
'H7 เปรียบเทียบกับการทำงานในออฟฟิศ จำนวนชั่วโมงในการทำงานของท่านเปลี่ยนแปลงไปอย่างไร':'Temp11',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [สภาพแวดล้อมการทำงานที่บ้านดีกว่าที่ทำงาน]':'Temp24',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ความสมดุลของงานและชีวิตส่วนตัว (work-life balance) ดี]':'Temp25',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [บริหารจัดการงานได้ดีขึ้น (เช่น การจัดลำดับความสำคัญของงาน การเลือกงาน)]':'Temp26',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาที่โฟกัสกับงานมากขึ้น]':'Temp27',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีการประชุมที่มีประสิทธิภาพขึ้น]':'Temp28',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีส่ิงรบกวนน้อยลง]':'Temp29',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ใช้เวลาเดินทางน้อยลง]':'Temp30',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาในการทำงานที่ยืดหยุ่นขึ้น]':'Temp31',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีกิจกรรมออกกำลังกายมากขึ้น]':'Temp32',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ได้หยุดพักระหว่างทำงานบ่อยขึ้น]':'Temp33',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีปัญหาสุขภาพน้อยลง]':'Temp34',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ใด้อยู่ใกล้ชิดกับครอบครัวมากขึ้น]':'Temp35',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีค่าใช้จ่ายน้อยลงในการเดินทาง อาหาร อื่นๆ]':'Temp36',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ได้สวมใส่เสื้อผ้าที่สบายมากขึ้น]':'Temp37',
'H8 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาในการทำงานให้เสร็จมากขึ้น]':'Temp38',
'H9 มีประโยชน์อื่นๆ ที่ท่านได้รับจากการทำงานที่บ้านแต่ไม่มีอยู่ในคำถามข้อที่แล้วหรือไม่?':'Temp39',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [สภาพแวดล้อมการทำงานที่บ้านแย่กว่าที่ทำงาน]':'Temp40',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีอุปกรณ์ hardware, monitor หรืออุปกรณ์ที่จำเป็นไม่เพียงพอ]':'Temp41',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ความสมดุลของงานและชีวิตส่วนตัว (work-life balance) ไม่ดี]':'Temp42',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีส่ิงรบกวนมากขึ้น]':'Temp43',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีปัญหาการเชื่อมต่อ (เช่น Internet, VPN, Remote desktop)]':'Temp44',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [รับรู้ว่าเพื่อนร่วมงานกำลังทำอะไรอยู่น้อยลง]':'Temp45',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [สื่อสารกับเพื่อนร่วมงานยาก]':'Temp46',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีความยากลำบากในการใช้งานเครื่องมือที่ช่วยทำงานร่วมกัน (collaboration tools)]':'Temp47',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่สามารถทำงานได้ เพราะต้องคอยงานจากเพื่อนร่วมงาน]':'Temp48',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีประชุมเยอะเกินไป]':'Temp49',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีกิจกรรมออกกำลังกายน้อยลง]':'Temp50',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ขาดการปฏิสัมพันธ์กับเพื่อนร่วมงาน]':'Temp51',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ได้หยุดพักระหว่างทำงานน้อยลง]':'Temp52',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ทำกิจกรรมประจำได้น้อยลง]':'Temp53',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ขาดเส้นแบ่งระหว่างงานและชีวิตส่วนตัว]':'Temp54',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีเวลาในการทำงานให้เสร็จน้อยลง]':'Temp55',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่มีคนดูแลบุตร]':'Temp56',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่มีกำลังใจในการทำงาน]':'Temp57',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีความลำบากในการหาอาหาร]':'Temp58',
'H10 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ที่ทำงานที่บ้านมี ergonomic ต่ำ]':'Temp59',
'H11 มีปัญหาหรือความลำบากอื่นๆ ที่ท่านได้ประสบจากการทำงานที่บ้านแต่ไม่มีอยู่ในคำถามข้อที่แล้วหรือไม่?':'Temp60',
#'H15 เครื่องมือซอฟต์แวร์ที่ช่วยในการทำงาน (software tools) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp61',
#'H16 เครื่องมือฮาร์ดแวร์ที่ช่วยในการทำงาน (hardware tools) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp62',
#'H17 กระบวนการทำงาน (process) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp63',
'H12 โดยรวม ท่านคิดว่าทำงานที่บ้านมีข้อดีกับท่านอย่างไร?':'Temp64',
'H13 โดยรวม ท่านคิดว่าทำงานที่บ้านมีข้อเสียกับท่านอย่างไร?':'Temp65',
'I1 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของคนส่วนใหญ่ที่ทำงานคล้ายๆ กับท่านเท่าไร?':'PerformanceOthers',
'I2 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วง 1 หรือ 2 ปีที่ผ่านมาเท่าไร?':'PerB1item',
'I3 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วงที่ท่านเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 เท่าไร?':'PerS1item',
'I4 มีสิ่งอื่นใดที่ท่านอยากให้เราทราบเกี่ยวกับประสิทธิผลในการทำงานหรือไม่':'Temp65.1',
'J1 เปรียบเทียบกับการทำงานในออฟฟิศ ประสิทธิผล (productivity) ของทีมท่านมีการเปลี่ยนแปลงไปอย่างไร?':'Temp66',
'J2 วัฒนธรรมการทำงานของทีมของท่านมีการเปลี่ยนแปลงไปอย่างไร?':'Temp67',
'J3 ทีมของท่านช่วยเหลือท่านอย่างไรบ้างในช่วงการแพร่ระบาดของ COVID-19?':'Temp68',
'J4 เมื่อเปรียบเทียบกับการทำงานในออฟฟิศ ท่านคิดว่าความสามารถของทีมในการทำงานเพื่อให้เสร็จสิ้นให้ทันเดดไลน์เปลี่ยนแปลงไปหรือไม่? หากใช่ โปรดอธิบาย':'Temp69',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการได้รับข้อมูลใดๆ จากเพื่อนร่วมทีม]':'Temp70',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การรับรู้ว่าเพื่อนร่วมทีมกำลังทำงานอะไรอยู่]':'Temp71',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ประสิทธิภาพของการสื่อสารกับเพื่อนร่วมทีม]':'Temp72',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความง่ายในการสื่อสารกับเพื่อนร่วมทีม]':'Temp73',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [งานที่ข้าพเจ้าทำให้กับทีม]':'Temp74',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การมีปฏิสัมพันธ์เชิงบวกกับเพื่อนร่วมทีม]':'Temp75',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การมีปฏิสัมพันธ์เชิงบวกกับหัวหน้างาน]':'Temp76',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการประชุมแบบประจำ]':'Temp77',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการประชุมแบบไม่ได้นัดมาก่อน]':'Temp78',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [คุณภาพของการประชุมแบบประจำ]':'Temp79',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความรู้สึกของการได้มีปฏิสัมพันธ์ทางสังคมกับทีม]':'Temp80',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การแบ่งปันความรู้ในทีม]':'Temp81',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการทานอาหาร หรือ coffee break ร่วมกันแบบ virtual]':'Temp82',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการพบปะเพื่อพูดคุยกันแบบ 1-1]':'Temp83',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [แบ่งปันภาพถ่ายขำขันหรือเรื่องตลก]':'Temp84',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [เล่นเกมส์ออนไลน์ด้วยกัน]':'Temp85',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการตั้ง online meeting channel เพื่อกิจกรรมสนุกสนาน]':'Temp86',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [เริ่มการประชุมหรือจบการประชุมด้วยเรื่องที่ไม่เกี่ยวกับงาน]':'Temp87',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีกิจกรรมอื่น ที่เพิ่มความสัมพันธ์ เช่น มีการนัดทำกิจกรรมร่วมกันออนไลน์]':'Temp88',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [ไม่มีที่กล่าวมาข้างตน]':'Temp89',
'J7 มีสิ่งอื่นใดที่ท่านอยากให้เราทราบเกี่ยวกับว่าการแพร่ระบาดของ COVID-19 มีผลกระทบกับท่านหรือทีมของท่านอย่างไร?':'Temp90',
'K1 มีเรื่องอื่นที่ท่านอยากแชร์เกี่ยวกับการทำงานที่บ้านหรือไม่?':'Temp91',
'Note':'Note'})

In [371]:
covid_df.columns.tolist()

['Timestamp',
 'Confirm_REMOVE',
 'Office_work_before_REMOVE',
 'RolesIncludeDeveloper',
 'YearsOfExperience',
 'YearsOfWorkFromHomeExperience',
 'Fulltime',
 'Coinhabitants',
 'Children',
 'Age',
 'Gender',
 'Education',
 'OrganizationSize',
 'Disabilities',
 'B1_REMOVE',
 'COVIDStatus0-5',
 'Isolation',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR7',
 'FR8',
 'FR9',
 'DP1',
 'DP2',
 'DP3',
 'DP4',
 'DP5',
 'Erg1',
 'Erg2',
 'Erg3',
 'Erg4',
 'Erg5',
 'Erg6',
 'OS1',
 'OS2',
 'OS3',
 'OS4',
 'OS5',
 'OS6',
 'OS7',
 'OS8',
 'OS9',
 'OS10',
 'OS11',
 'OS13',
 'OS14',
 'OS15',
 'OS16',
 'OS17',
 'OS18',
 'OS19',
 'OS20',
 'OS21',
 'OS22',
 'Temp1',
 'Temp2',
 'WHO5B1',
 'WHO5B2',
 'WHO5B3',
 'WHO5B4',
 'WHO5B5',
 'HPQB1',
 'HPQB2',
 'HPQB3',
 'HPQB4',
 'HPQB6',
 'HPQB7',
 'HPQB8',
 'Temp3',
 'Temp5',
 'Temp6',
 'Temp7',
 'WHO5S1',
 'WHO5S2',
 'WHO5S3',
 'WHO5S4',
 'WHO5S5',
 'HPQS1',
 'HPQS2',
 'HPQS3',
 'HPQS4',
 'HPQS6',
 'HPQS7',
 'HPQS8',
 'Temp7.1',
 'Temp9',
 'Temp10',
 'Temp11',
 

In [372]:
original_covid_df.columns.tolist()

['Timestamp',
 'โปรดยืนยันว่าท่านได้อ่านและเข้าใจเนื้อหาในเอกสารชี้แจงผู้เข้าร่วมวิจัย โดยการตอบแบบสอบถาม (Self-Administered Questionnaire Participant Information Sheet) แล้ว',
 'ท่านเคยทำงานในที่ทำงาน (ออฟฟิศ) อย่างน้อยเป็นเวลา 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน',
 'A1 ตำแหน่งงานปัจจุบันของท่าน',
 'A2 ท่านมีประสบการณ์ในการทำงานด้านพัฒนาซอฟต์แวร์มาแล้วเป็นเวลากี่ปี? ',
 'A3 ท่านมีประสบการณ์ในการทำงานที่บ้านมาแล้วเป็นเวลากี่ปี?',
 'A4 ตำแหน่งงานหลักของข้าพเจ้าเป็นแบบ',
 'A5 ท่านมีคนอาศัยอยู่ในบ้านเดียวกันกับท่านจำนวนกี่คน (รวมตัวท่านเอง)?',
 'A6 คนที่อาศัยอยู่ในบ้านเดียวกับท่าน มีกี่คนที่อายุน้อยกว่า 12 ปี?',
 'A7 ท่านอายุเท่าไหร่?',
 'A8 โปรดระบุเพศของท่าน',
 'A9 ท่านจบการศึกษาสูงสุดในระดับใด?',
 'A10 บริษัทของท่าน (ไม่ใช่ทีมของท่าน) มีจำนวนพนักงานประมาณกี่คน?',
 'A11 ท่านมีความพิการทางร่างกายที่มีผลต่อการทำงานของท่านหรือไม่?',
 'B1 โปรดเลือกข้อที่เหมาะสมกับคุณที่สุด',
 'C1 โปรดเลือกทุกข้อที่ตรงกับสถานะของท่าน',
 'C2 เลือกข้อที่ตรงกับท่านมากที่สุด',
 'D1 ท่า

In [373]:
covid_df.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [374]:
# reorder column

covid_df = covid_df.reindex(columns=['Timestamp',
 'Confirm_REMOVE',
 'Office_work_before_REMOVE',
 'Language',
 'AtLeastOneOpenResponse',
 'RolesIncludeDeveloper',
 'YearsOfExperience',
 'YearsOfWorkFromHomeExperience',
 'Fulltime',
 'Coinhabitants',
 'Children',
 'Age',
 'Gender',
 'Education',
 'OrganizationSize',
 'Disabilities',
 'B1_REMOVE',
 'COVIDStatus0-5',
 'Isolation',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR6',
 'FR7',
 'FR8',
 'FR9',
 'DP1',
 'DP2',
 'DP3',
 'DP4',
 'DP5',
 'Erg1',
 'Erg2',
 'Erg3',
 'Erg4',
 'Erg5',
 'Erg6',
 'OS1',
 'OS2',
 'OS3',
 'OS4',
 'OS5',
 'OS6',
 'OS7',
 'OS8',
 'OS9',
 'OS10',
 'OS11',
 'OS13',
 'OS12',
 'OS14',
 'OS15',
 'OS16',
 'OS17',
 'OS18',
 'OS19',
 'OS20',
 'OS21',
 'OS22',
 'WHO5B1',
 'WHO5B2',
 'WHO5B3',
 'WHO5B4',
 'WHO5B5',
 'HPQB1',
 'HPQB2',
 'HPQB3',
 'HPQB4',
 'HPQB5',
 'HPQB6',
 'HPQB7',
 'HPQB8',
 'WHO5S1',
 'WHO5S2',
 'WHO5S3',
 'WHO5S4',
 'WHO5S5',
 'HPQS1',
 'HPQS2',
 'HPQS3',
 'HPQS4',
 'HPQS5',
 'HPQS6',
 'HPQS7',
 'HPQS8',
 'PerformanceOthers',
 'PerB1item',
 'PerS1item',
 'Temp1',
 'Temp2',
 'Temp3',
 'Temp5',
 'Temp6',
 'Temp7',
 'Temp7.1',
 'Temp9',
 'Temp10',
 'Temp11',
 'Temp24',
 'Temp25',
 'Temp26',
 'Temp27',
 'Temp28',
 'Temp29',
 'Temp30',
 'Temp31',
 'Temp32',
 'Temp33',
 'Temp34',
 'Temp35',
 'Temp36',
 'Temp37',
 'Temp38',
 'Temp39',
 'Temp40',
 'Temp41',
 'Temp42',
 'Temp43',
 'Temp44',
 'Temp45',
 'Temp46',
 'Temp47',
 'Temp48',
 'Temp49',
 'Temp50',
 'Temp51',
 'Temp52',
 'Temp53',
 'Temp54',
 'Temp55',
 'Temp56',
 'Temp57',
 'Temp58',
 'Temp59',
 'Temp60',
 'Temp64',
 'Temp65',
 'Temp65.1',
 'Temp66',
 'Temp67',
 'Temp68',
 'Temp69',
 'Temp70',
 'Temp71',
 'Temp72',
 'Temp73',
 'Temp74',
 'Temp75',
 'Temp76',
 'Temp77',
 'Temp78',
 'Temp79',
 'Temp80',
 'Temp81',
 'Temp82',
 'Temp83',
 'Temp84',
 'Temp85',
 'Temp86',
 'Temp87',
 'Temp88',
 'Temp90',
 'Temp91',
 ])

In [375]:
covid_df.shape

(167, 165)

# questionnaire verion 1

In [376]:
covid_df_v1 = pd.read_excel('แบบสอบถามเรื่องผลกระทบจาก COVID-19 กับนักพัฒนาซอฟตแวร์ (Responses)-1st_round.xlsx')
original_covid_df_v1 = covid_df_v1.copy()

In [377]:
covid_df_v1.shape

(88, 179)

In [378]:
covid_df_v1 = covid_df_v1.rename(columns={'โปรดยืนยันว่าท่านได้อ่านและเข้าใจเนื้อหาในเอกสารชี้แจงผู้เข้าร่วมวิจัย โดยการตอบแบบสอบถาม (Self-Administered Questionnaire Participant Information Sheet) แล้ว':'Confirm_REMOVE',
'ท่านเคยทำงานในที่ทำงาน (ออฟฟิศ) อย่างน้อยเป็นเวลา 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน':'Office_work_before_REMOVE',
'A1 ตำแหน่งงานปัจจุบันของท่าน':'RolesIncludeDeveloper',
'A2 ท่านมีประสบการณ์ในการทำงานด้านพัฒนาซอฟต์แวร์มาแล้วเป็นเวลากี่ปี? ':'YearsOfExperience',
'A3 ท่านมีประสบการณ์ในการทำงานที่บ้านมาแล้วเป็นเวลากี่ปี?':'YearsOfWorkFromHomeExperience',
'A4 ตำแหน่งงานหลักของข้าพเจ้าเป็นแบบ':'Fulltime',
'A5 ท่านมีคนอาศัยอยู่ในบ้านเดียวกันกับท่านจำนวนกี่คน (รวมตัวท่านเอง)?':'Coinhabitants',
'A6 คนที่อาศัยอยู่ในบ้านเดียวกับท่าน มีกี่คนที่อายุน้อยกว่า 12 ปี?':'Children',
'A7 ท่านอายุเท่าไหร่?':'Age',
'A8 โปรดระบุเพศของท่าน':'Gender',
'A9 ท่านจบการศึกษาสูงสุดในระดับใด?':'Education',
'A10 บริษัทของท่าน (ไม่ใช่ทีมของท่าน) มีจำนวนพนักงานประมาณกี่คน?':'OrganizationSize',
'A11 ท่านมีความพิการทางร่างกายที่มีผลต่อการทำงานของท่านหรือไม่?':'Disabilities',
'B1 โปรดเลือกข้อที่เหมาะสมกับคุณที่สุด':'B1_REMOVE',
'C1 โปรดเลือกทุกข้อที่ตรงกับสถานะของท่าน':'COVIDStatus0-5',
'C2 เลือกข้อที่ตรงกับท่านมากที่สุด':'Isolation',
'D1 ท่านรู้สึกกลัวว่า [ท่านจะติดเชื้อ COVID-19]':'FR1',
'D1 ท่านรู้สึกกลัวว่า [ท่านจะเสียชีวิตจากการติดเชื้อ COVID-19]':'FR2',
'D1 ท่านรู้สึกกลัวว่า [คนในครอบครัวของท่านจะเสียชีวิตจากการติดเชื้อ COVID-19]':'FR3',
'D2 ตอนนี้ท่านรู้สึก [หวาดกลัว]':'FR4',
'D2 ตอนนี้ท่านรู้สึก [ตนเองไม่สามารถทำอะไรเพื่อแก้ปัญหาได้]':'FR5',
'D2 ตอนนี้ท่านรู้สึก [ตกใจกลัวอย่างที่สุด]':'FR6',
'D3 โปรดตามคำถามต่อไปนี้ [ท่านหวาดกลัวว่าท่านจะไม่มีเงินหากท่านไม่สามารถทำงานได้ในช่วง 2-3 เดือนข้างหน้าหรือไม่?]':'FR7',
'D3 โปรดตามคำถามต่อไปนี้ [ปกติแล้วท่านสามารถหายจากอาการเจ็บป่วยได้รวดเร็วหรือไม่?]':'FR8',
'D3 โปรดตามคำถามต่อไปนี้ [ท่านมีเพื่อนที่สามารถติดต่อได้ทางโทรศัพท์ อีเมล หรืออื่นๆ หรือไม่ ในเวลาที่ท่านรู้สึกโดดเดี่ยว]':'FR9',
'How many different prescription medications are you on?':'FR10',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าปรึกษากับคนอื่นเกี่ยวกับข้อมูลเรื่อง COVID-19 ที่ข้าพเจ้าได้รับมา]':'DP1',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้ามีชุดยังชีพฉุกเฉิน]':'DP2',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าได้วางแผนไว้แล้วว่าต้องปฏิบัติตัวอย่างไรหากข้าพเจ้าตรวจพบเชื้อ COVID-19]':'DP3',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้าปฏิบัติตามคำแนะนำของหน่วยงานของรัฐเกี่ยวกับการปฏิบัติตัวช่วง COVID-19 อย่างเคร่งครัด]':'DP4',
'E1 โปรดตอบคำถามต่อไปนี้ [ข้าพเจ้ารู้จักคนที่จะช่วยข้าพเจ้าได้หากข้าพเจ้าจะต้องกักตัว]':'DP5',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [สถาพแวดล้อมของที่ทำงานที่บ้านของข้าพเจ้าปราศจากสิ่งรบกวน]':'Erg1',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [ระดับของเสียงรบกวนที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่ยอมรับได้ (ไม่รบกวนการทำงาน)]':'Erg2',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [อุณหภูมิของที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่เหมาะสม]':'Erg3',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [แสงในที่ทำงานที่บ้านของข้าพเจ้าอยู่ในระดับที่เหมาะสม]':'Erg4',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [เก้าอี้ทำงานของข้าพเจ้ามีความสบาย]':'Erg5',
'E2 โปรดระบุว่าท่านเห็นด้วยกับคำถามเกี่ยวกับการทำงานที่บ้านต่อไปนี้ มากน้อยแค่ไหน [ที่ทำงานที่บ้านของข้าพเจ้ามีความเหมาะสมกับการทำงาน]':'Erg6',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ข้าพเจ้าสามารถนำอุปกรณ์ (เช่น จอภาพ) จากที่ทำงานกลับมาที่บ้านได้]':'OS1',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะซื้ออุปกรณ์ใหม่ให้พนักงานเพื่อทำงานที่บ้าน]':'OS2',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะซื้อซอฟต์แวร์ที่จำเป็นสำหรับการทำงานที่บ้าน]':'OS3',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าจะจ่ายค่าอินเตอร์เน็ตให้ข้าพเจ้า]':'OS4',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าจะไม่ถูกไล่ออก]':'OS5',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าจะได้รับเงินเดือนเหมือนเดิม]':'OS6',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าข้าพเจ้าสามารถลาหยุดหากข้าพเจ้าป่วยหรือลาเพื่อไปดูแลคนในครอบครัวได้]':'OS7',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ายืนยันกับข้าพเจ้าว่าองค์กรมีความเข้าใจหากผลการทำงานของข้าพเจ้าแย่ลง]':'OS8',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ายังคงมีการประชุมประจำเช่นเดิม (เช่น ผ่านระบบ video call)]':'OS9',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการประชุม (เช่น ผ่าน video call) น้อยลง]':'OS10',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าให้กำลังใจพนักงาน โดยให้มาพบเจอกัน (แบบออนไลน์) เป็นประจำ]':'OS11',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการสร้างกิจกรรม virtual social events (ผ่านระบบ via video chat) ให้ทุกคนในทีม]':'OS12',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ในแต่ละวัน ข้าพเจ้าทำงานโดยเปิด video call หรือ audio call ค้างไว้กับเพื่อนร่วมงานในทีม]':'OS13',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าส่งอาหารให้กับพนักงานที่บ้าน]':'OS14',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ามีกิจกรรมให้บุตรของข้าพเจ้าได้ทำระหว่างที่ข้าพเจ้าทำงาน]':'OS15',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ามีกิจกรรมให้พนักงานออกกำลังกายที่บ้าน]':'OS16',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้าสนับสนุนให้พนักงานใช้เวลาช่วงนี้ในการฝึกฝนตนเอง (professional training)]':'OS17',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีระบบที่รองรับการทำงานที่บ้านได้อย่างดี (เช่น source control, VPN, remote desktop, file sharing)]':'OS18',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้ามีการตรวจเช็ค commits, change request หรือ pull request (peer code review)]':'OS19',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [ทีมของข้าพเจ้าใช้ระบบ build system เพื่อทำการ compile และ test software]':'OS20',
'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [องค์กรของข้าพเจ้ากำลังปรับปรุงเอกสารและกระบวนการทำงาน (เช่น การ approve change request/pull request)]':'OS21',

'F1 ข้อความด้านล่างนี้อธิบายสิ่งที่องค์กร (บริษัทของท่าน) อาจจะช่วยสนับสนุนท่านระหว่างการทำงานที่บ้าน โปรดเลือกคำตอบโดยใช้วิธีการดังที่อธิบายไว้ด้านบน ในแต่ละข้อ [มีคนในองค์กรที่ช่วยจัดลำดับความสำคัญของงานและตรวจเช็คข้อมูลใน backlog ให้ถูกต้องเสมอ]':'OS22',
'F2 มีสิ่งอื่นใดที่บริษัทของท่านกำลังทำหรือควรจะทำเพื่อช่วยท่านระหว่างทำงานที่บ้าน?':'Temp1',
'F3 มีสิ่งใดที่บริษัทของท่านทำแล้วท่านคิดว่าทำให้การทำงานที่บ้านลำบากมากขึ้น?':'Temp2',

'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกสดชื่นและมีกำลังใจดี]':'WHO5B1',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกสงบและผ่อนคลาย]':'WHO5B2',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้ารู้สึกตื่นตัวและกระฉับกระเฉง]':'WHO5B3',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [ข้าพเจ้าตื่นมาแล้วรู้สึกสดชื่นและพักผ่อนอย่างเพียงพอ]':'WHO5B4',
'G1 เมื่อข้าพเจ้านึกถึงเวลาช่วง 4 สัปดาห์​ (หรือ 1 เดือน)​ ก่อนที่ข้าพเจ้าจะเริ่มทำงานที่บ้าน … [กิจกรรมในแต่ละวันของข้าพเจ้าเต็มไปด้วยส่ิงที่น่าสนใจ]':'WHO5B5',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ประสิทธิภาพการทำงานของท่านสูงกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQB1',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ประสิทธิภาพการทำงานของท่านต่ำกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQB2',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านเลือกที่จะไม่ทำงานทั้งที่ควรจะต้องทำงานนั้นๆ บ่อยแค่ไหน?]':'HPQB3',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านทำงานไม่ละเอียดเท่าที่ควรบ่อยแค่ไหน?]':'HPQB4',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านทำงานได้คุณภาพต่ำกว่าที่ควรจะเป็นบ่อยแค่ไหน?]':'HPQB5',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [ท่านไม่มีสมาธิกับการทำงานบ่อยแค่ไหน?]':'HPQB6',
'G2 ในช่วง 4 สัปดาห์ก่อนที่ท่านจะเริ่มทำงานที่บ้าน … [บ่อยแค่ไหนที่ความกังวลเกี่ยวกับสถานการณ์ของ COVID-19 ทำให้ท่านทำงานได้น้อยลง?]':'HPQB7',
'G3 หากท่านเปรียบเทียบประสิทธิภาพในการทำงานของท่านกับคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน ในช่วง 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน ท่านคิดว่าอย่างไร?':'HPQB8',
'G4 ตอนที่ท่านทำงานที่ออฟฟิศ ในบริเวณโต๊ะที่ท่านทำงานมีคนจำนวนกี่คน (รวมตัวท่านเอง)?':'Temp3',
'G5 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านมีประชุมวันละประมาณกี่ครั้ง?':'Temp4',
#'G5 ตอนที่ท่านทำงานที่ออฟฟิศ ทีมของท่านมีการประชุมแบบ stand-up meeting เป็นประจำหรือไม่?':'Temp5',
'G6 ตอนที่ท่านทำงานที่ออฟฟิศ ทีมของท่านมีการประชุมแบบ stand-up meeting เป็นประจำหรือไม่?':'Temp5',    #v1
#'G6 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านทำงานร่วมกับเพื่อนร่วมงานอีกประมาณกี่คน?':'Temp6',
'G7 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านทำงานร่วมกับเพื่อนร่วมงานอีกประมาณกี่คน?':'Temp6',    #v1
#'G7 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านใช้เวลาเดินทาง (นาที) โดยเฉลี่ยเท่าไหร่ต่อวัน โปรดรวมระยะเวลาทั้งขาไปและขากลับ':'Temp7',
'G8 ตอนที่ท่านทำงานที่ออฟฟิศ ท่านใช้เวลาเดินทาง (นาที) โดยเฉลี่ยเท่าไหร่ต่อวัน โปรดรวมระยะเวลาทั้งขาไปและขากลับ':'Temp7', #v1
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกสดชื่นและมีกำลังใจดี]':'WHO5S1',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกสงบและผ่อนคลาย]':'WHO5S2',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้ารู้สึกตื่นตัวและกระฉับกระเฉง]':'WHO5S3',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [ข้าพเจ้าตื่นมาแล้วรู้สึกสดชื่นและพักผ่อนอย่างเพียงพอ]':'WHO5S4',
'H1 ตั้งแต่ข้าพเจ้าเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 [กิจกรรมในแต่ละวันของข้าพเจ้าเต็มไปด้วยส่ิงที่น่าสนจ]':'WHO5S5',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ประสิทธิภาพการทำงานของท่านสูงกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQS1',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ประสิทธิภาพการทำงานของท่านต่ำกว่าเพื่อนร่วมงานส่วนใหญ่ที่ทำงานด้วยกันบ่อยแค่ไหน?]':'HPQS2',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านไม่ทำงานทั้งที่ควรจะต้องทำงานบ่อยแค่ไหน?]':'HPQS3',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านทำงานไม่ละเอียดเท่าที่ควรบ่อยแค่ไหน?]':'HPQS4',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านทำงานได้คุณภาพต่ำกว่าที่ควรจะเป็นบ่อยแค่ไหน?]':'HPQS5',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [ท่านไม่มีสมาธิกับการทำงานบ่อยแค่ไหน?]':'HPQS6',
'H2 ตั้งแต่เร่ิมทำงานที่บ้าน [บ่อยแค่ไหนที่ความกังวลเกี่ยวกับสถานการณ์ของ COVID-19 ทำให้ท่านทำงานได้น้อยลง?]':'HPQS7',
'H3 หากท่านเปรียบเทียบประสิทธิภาพในการทำงานของท่านกับคนส่วนใหญ่ที่ทำงานคล้ายๆ กันกับท่าน ในช่วง 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ตั้งแต่ท่านจะเริ่มทำงานที่บ้าน ท่านคิดว่าอย่างไร?':'HPQS8',
'H4 ตอนที่ท่านทำงานที่บ้าน ทีมของท่านมีการประชุมแบบ stand-up meeting เป็นประจำหรือไม่?':'Temp7.1',
'H5 ท่านทำงานที่บ้านเป็นประจำหรือไม่ก่อนเกิดการแพร่ระบาดของ COVID-19?':'Temp8',
#'H5 ก่อนที่ท่านจะเริ่มทำงานที่บ้าน บุตรของท่านไปเรียนที่โรงเรียนหรือเนิร์สเซอรี่หรือไม่':'Temp9',
'H6 ก่อนที่ท่านจะเริ่มทำงานที่บ้าน บุตรของท่านไปเรียนที่โรงเรียนหรือเนิร์สเซอรี่หรือไม่':'Temp9',   #v1
#'H6 การที่ต้องดูแลบุตรของท่านระหว่างการแพร่ระบาดของ COVID-19 เป็นเรื่องง่ายหรือยากแค่ไหนสำหรับท่าน?':'Temp10',
'H7 การที่ต้องดูแลบุตรของท่านระหว่างการแพร่ระบาดของ COVID-19 เป็นเรื่องง่ายหรือยากแค่ไหนสำหรับท่าน?':'Temp10',   #v1
'H8 โปรดอธิบายว่าสถานที่ทำงานที่บ้านของท่านเป็นอย่างไร ':'Temp11',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Laptop]':'Temp12',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Desktop]':'Temp13',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Multiple screens]':'Temp14',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Headphones]':'Temp15',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Keyboard]':'Temp16',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Mouse]':'Temp17',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [โต๊ะ]':'Temp18',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [Whiteboard]':'Temp19',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [โต๊ะแบบ Ergonomic]':'Temp20',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [เก้าอี้แบบ Ergonomic]':'Temp21',
'H9 โปรดอธิบายว่าสภาพแวดล้อมในการทำงานของท่านที่บ้านเป็นอย่างไร [แสงสว่างตามธรรมชาติ]':'Temp22',
#'H7 เปรียบเทียบกับการทำงานในออฟฟิศ จำนวนชั่วโมงในการทำงานของท่านเปลี่ยนแปลงไปอย่างไร':'Temp11',
'H10 เปรียบเทียบกับการทำงานในออฟฟิศ จำนวนชั่วโมงในการทำงานของท่านเปลี่ยนแปลงไปอย่างไร':'Temp11.1',   #v1
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [สภาพแวดล้อมการทำงานที่บ้านดีกว่าที่ทำงาน]':'Temp24',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ความสมดุลของงานและชีวิตส่วนตัว (work-life balance) ดี]':'Temp25',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [บริหารจัดการงานได้ดีขึ้น (เช่น การจัดลำดับความสำคัญของงาน การเลือกงาน)]':'Temp26',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาที่โฟกัสกับงานมากขึ้น]':'Temp27',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีการประชุมที่มีประสิทธิภาพขึ้น]':'Temp28',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีส่ิงรบกวนน้อยลง]':'Temp29',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ใช้เวลาเดินทางน้อยลง]':'Temp30',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาในการทำงานที่ยืดหยุ่นขึ้น]':'Temp31',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีกิจกรรมออกกำลังกายมากขึ้น]':'Temp32',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ได้หยุดพักระหว่างทำงานบ่อยขึ้น]':'Temp33',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีปัญหาสุขภาพน้อยลง]':'Temp34',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ใด้อยู่ใกล้ชิดกับครอบครัวมากขึ้น]':'Temp35',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีค่าใช้จ่ายน้อยลงในการเดินทาง อาหาร อื่นๆ]':'Temp36',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [ได้สวมใส่เสื้อผ้าที่สบายมากขึ้น]':'Temp37',
'H11 โปรดตอบคำถามด้านล่างเกี่ยวกับประโยชน์ที่ท่านคิดว่าได้รับจากการทำงานที่บ้านและความสำคัญของประโยชน์เหล่านั้น [มีเวลาในการทำงานให้เสร็จมากขึ้น]':'Temp38',
'H12 มีประโยชน์อื่นๆ ที่ท่านได้รับจากการทำงานที่บ้านแต่ไม่มีอยู่ในคำถามข้อที่แล้วหรือไม่?':'Temp39',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [สภาพแวดล้อมการทำงานที่บ้านแย่กว่าที่ทำงาน]':'Temp40',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีอุปกรณ์ hardware, monitor หรืออุปกรณ์ที่จำเป็นไม่เพียงพอ]':'Temp41',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ความสมดุลของงานและชีวิตส่วนตัว (work-life balance) ไม่ดี]':'Temp42',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีส่ิงรบกวนมากขึ้น]':'Temp43',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีปัญหาการเชื่อมต่อ (เช่น Internet, VPN, Remote desktop)]':'Temp44',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [รับรู้ว่าเพื่อนร่วมงานกำลังทำอะไรอยู่น้อยลง]':'Temp45',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [สื่อสารกับเพื่อนร่วมงานยาก]':'Temp46',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีความยากลำบากในการใช้งานเครื่องมือที่ช่วยทำงานร่วมกัน (collaboration tools)]':'Temp47',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่สามารถทำงานได้ เพราะต้องคอยงานจากเพื่อนร่วมงาน]':'Temp48',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีประชุมเยอะเกินไป]':'Temp49',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีกิจกรรมออกกำลังกายน้อยลง]':'Temp50',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ขาดการปฏิสัมพันธ์กับเพื่อนร่วมงาน]':'Temp51',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ได้หยุดพักระหว่างทำงานน้อยลง]':'Temp52',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ทำกิจกรรมประจำได้น้อยลง]':'Temp53',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ขาดเส้นแบ่งระหว่างงานและชีวิตส่วนตัว]':'Temp54',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีเวลาในการทำงานให้เสร็จน้อยลง]':'Temp55',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่มีคนดูแลบุตร]':'Temp56',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ไม่มีกำลังใจในการทำงาน]':'Temp57',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [มีความลำบากในการหาอาหาร]':'Temp58',
'H13 โปรดตอบคำถามด้านล่างเกี่ยวกับปัญหาหรือความลำบากที่ท่านประสบจากการทำงานที่บ้านและผลกระทบของปัญหาหรือความลำบากเหล่านั้น [ที่ทำงานที่บ้านมี ergonomic ต่ำ]':'Temp59',
'H14 มีปัญหาหรือความลำบากอื่นๆ ที่ท่านได้ประสบจากการทำงานที่บ้านแต่ไม่มีอยู่ในคำถามข้อที่แล้วหรือไม่?':'Temp60',
'H15 เครื่องมือซอฟต์แวร์ที่ช่วยในการทำงาน (software tools) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp61',
'H16 เครื่องมือฮาร์ดแวร์ที่ช่วยในการทำงาน (hardware tools) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp62',
'H17 กระบวนการทำงาน (process) ใดที่ท่านพบว่ามีประโยชน์ระหว่างการทำงานที่บ้าน?':'Temp63',
'H18 โดยรวม ท่านคิดว่าทำงานที่บ้านมีข้อดีกับท่านอย่างไร?':'Temp64',
'H19 โดยรวม ท่านคิดว่าทำงานที่บ้านมีข้อเสียกับท่านอย่างไร?':'Temp65',
'I1 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของคนส่วนใหญ่ที่ทำงานคล้ายๆ กับท่านเท่าไร?':'PerformanceOthers',
'I2 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วง 1 หรือ 2 ปีที่ผ่านมาเท่าไร?':'PerB1item',
'I3 ท่านจะให้คะแนนของประสิทธิผลทั่วไปของตนเองในช่วงที่ท่านเริ่มทำงานที่บ้านเนื่องจากการแพร่ระบาดของ COVID-19 เท่าไร?':'PerS1item',
'I4 มีสิ่งอื่นใดที่ท่านอยากให้เราทราบเกี่ยวกับประสิทธิผลในการทำงานหรือไม่':'Temp65.1',
'J1 เปรียบเทียบกับการทำงานในออฟฟิศ ประสิทธิผล (productivity) ของทีมท่านมีการเปลี่ยนแปลงไปอย่างไร?':'Temp66',
'J2 วัฒนธรรมการทำงานของทีมของท่านมีการเปลี่ยนแปลงไปอย่างไร?':'Temp67',
'J3 ทีมของท่านช่วยเหลือท่านอย่างไรบ้างในช่วงการแพร่ระบาดของ COVID-19?':'Temp68',
'J4 เมื่อเปรียบเทียบกับการทำงานในออฟฟิศ ท่านคิดว่าความสามารถของทีมในการทำงานเพื่อให้เสร็จสิ้นให้ทันเดดไลน์เปลี่ยนแปลงไปหรือไม่? หากใช่ โปรดอธิบาย':'Temp69',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการได้รับข้อมูลใดๆ จากเพื่อนร่วมทีม]':'Temp70',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การรับรู้ว่าเพื่อนร่วมทีมกำลังทำงานอะไรอยู่]':'Temp71',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ประสิทธิภาพของการสื่อสารกับเพื่อนร่วมทีม]':'Temp72',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความง่ายในการสื่อสารกับเพื่อนร่วมทีม]':'Temp73',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [งานที่ข้าพเจ้าทำให้กับทีม]':'Temp74',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การมีปฏิสัมพันธ์เชิงบวกกับเพื่อนร่วมทีม]':'Temp75',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การมีปฏิสัมพันธ์เชิงบวกกับหัวหน้างาน]':'Temp76',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการประชุมแบบประจำ]':'Temp77',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความถี่ในการประชุมแบบไม่ได้นัดมาก่อน]':'Temp78',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [คุณภาพของการประชุมแบบประจำ]':'Temp79',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [ความรู้สึกของการได้มีปฏิสัมพันธ์ทางสังคมกับทีม]':'Temp80',
'J5 การสื่อสารกับเพื่อนร่วมทีมของท่านเปลี่ยนไปอย่างไรหลังจากเริ่มทำงานที่บ้าน [การแบ่งปันความรู้ในทีม]':'Temp81',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการทานอาหาร หรือ coffee break ร่วมกันแบบ virtual]':'Temp82',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการพบปะเพื่อพูดคุยกันแบบ 1-1]':'Temp83',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [แบ่งปันภาพถ่ายขำขันหรือเรื่องตลก]':'Temp84',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [เล่นเกมส์ออนไลน์ด้วยกัน]':'Temp85',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีการตั้ง online meeting channel เพื่อกิจกรรมสนุกสนาน]':'Temp86',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [เริ่มการประชุมหรือจบการประชุมด้วยเรื่องที่ไม่เกี่ยวกับงาน]':'Temp87',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [มีกิจกรรมอื่น ที่เพิ่มความสัมพันธ์ เช่น มีการนัดทำกิจกรรมร่วมกันออนไลน์]':'Temp88',
'J6 ท่านมีการติดต่อทางสังคมกับเพื่อนร่วมงานอย่างไร? (จากเดิมที่อาจจะเคยไปคุยกันเมื่อพบปะ หรือท่านอาหารร่วมกัน) [ไม่มีที่กล่าวมาข้างตน]':'Temp89',
'J7 มีสิ่งอื่นใดที่ท่านอยากให้เราทราบเกี่ยวกับว่าการแพร่ระบาดของ COVID-19 มีผลกระทบกับท่านหรือทีมของท่านอย่างไร?':'Temp90',
'K1 มีเรื่องอื่นที่ท่านอยากแชร์เกี่ยวกับการทำงานที่บ้านหรือไม่?':'Temp91',
'Note':'Note'})

In [379]:
covid_df_v1.columns.tolist()

['Timestamp',
 'Email Address',
 'Confirm_REMOVE',
 'Office_work_before_REMOVE',
 'RolesIncludeDeveloper',
 'YearsOfExperience',
 'YearsOfWorkFromHomeExperience',
 'Fulltime',
 'Coinhabitants',
 'Children',
 'Age',
 'Gender',
 'Education',
 'OrganizationSize',
 'Disabilities',
 'B1_REMOVE',
 'COVIDStatus0-5',
 'Isolation',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR7',
 'FR8',
 'FR9',
 'DP1',
 'DP2',
 'DP3',
 'DP4',
 'DP5',
 'Erg1',
 'Erg2',
 'Erg3',
 'Erg4',
 'Erg5',
 'Erg6',
 'OS1',
 'OS2',
 'OS3',
 'OS4',
 'OS5',
 'OS6',
 'OS7',
 'OS8',
 'OS9',
 'OS10',
 'OS11',
 'OS13',
 'OS14',
 'OS15',
 'OS16',
 'OS17',
 'OS18',
 'OS19',
 'OS20',
 'OS21',
 'OS22',
 'Temp1',
 'Temp2',
 'WHO5B1',
 'WHO5B2',
 'WHO5B3',
 'WHO5B4',
 'WHO5B5',
 'HPQB1',
 'HPQB2',
 'HPQB3',
 'HPQB4',
 'HPQB6',
 'HPQB7',
 'HPQB8',
 'Temp3',
 'Temp4',
 'Temp5',
 'Temp6',
 'Temp7',
 'WHO5S1',
 'WHO5S2',
 'WHO5S3',
 'WHO5S4',
 'WHO5S5',
 'HPQS1',
 'HPQS2',
 'HPQS3',
 'HPQS4',
 'HPQS6',
 'HPQS7',
 'HPQS8',
 'Temp7.1',
 'Tem

In [380]:
original_covid_df_v1.columns.tolist()

['Timestamp',
 'Email Address',
 'โปรดยืนยันว่าท่านได้อ่านและเข้าใจเนื้อหาในเอกสารชี้แจงผู้เข้าร่วมวิจัย โดยการตอบแบบสอบถาม (Self-Administered Questionnaire Participant Information Sheet) แล้ว',
 'ท่านเคยทำงานในที่ทำงาน (ออฟฟิศ) อย่างน้อยเป็นเวลา 4 สัปดาห์ (28 วัน หรือ 1 เดือน) ก่อนที่ท่านจะเริ่มทำงานที่บ้าน',
 'A1 ตำแหน่งงานปัจจุบันของท่าน',
 'A2 ท่านมีประสบการณ์ในการทำงานด้านพัฒนาซอฟต์แวร์มาแล้วเป็นเวลากี่ปี? ',
 'A3 ท่านมีประสบการณ์ในการทำงานที่บ้านมาแล้วเป็นเวลากี่ปี?',
 'A4 ตำแหน่งงานหลักของข้าพเจ้าเป็นแบบ',
 'A5 ท่านมีคนอาศัยอยู่ในบ้านเดียวกันกับท่านจำนวนกี่คน (รวมตัวท่านเอง)?',
 'A6 คนที่อาศัยอยู่ในบ้านเดียวกับท่าน มีกี่คนที่อายุน้อยกว่า 12 ปี?',
 'A7 ท่านอายุเท่าไหร่?',
 'A8 โปรดระบุเพศของท่าน',
 'A9 ท่านจบการศึกษาสูงสุดในระดับใด?',
 'A10 บริษัทของท่าน (ไม่ใช่ทีมของท่าน) มีจำนวนพนักงานประมาณกี่คน?',
 'A11 ท่านมีความพิการทางร่างกายที่มีผลต่อการทำงานของท่านหรือไม่?',
 'B1 โปรดเลือกข้อที่เหมาะสมกับคุณที่สุด',
 'C1 โปรดเลือกทุกข้อที่ตรงกับสถานะของท่าน',
 'C2 เลือกข้อที่ตรงกับท่านมา

In [381]:
covid_df_v1.columns[covid_df_v1.columns.duplicated()]

Index([], dtype='object')

In [382]:
# reorder column

covid_df_v1 = covid_df_v1.reindex(columns=['Timestamp',
 'Confirm_REMOVE',
 'Office_work_before_REMOVE',
 'Language',
 'AtLeastOneOpenResponse',
 'RolesIncludeDeveloper',
 'YearsOfExperience',
 'YearsOfWorkFromHomeExperience',
 'Fulltime',
 'Coinhabitants',
 'Children',
 'Age',
 'Gender',
 'Education',
 'OrganizationSize',
 'Disabilities',
 'B1_REMOVE',
 'COVIDStatus0-5',
 'Isolation',
 'FR1',
 'FR2',
 'FR3',
 'FR4',
 'FR5',
 'FR6',
 'FR7',
 'FR8',
 'FR9',
 'DP1',
 'DP2',
 'DP3',
 'DP4',
 'DP5',
 'Erg1',
 'Erg2',
 'Erg3',
 'Erg4',
 'Erg5',
 'Erg6',
 'OS1',
 'OS2',
 'OS3',
 'OS4',
 'OS5',
 'OS6',
 'OS7',
 'OS8',
 'OS9',
 'OS10',
 'OS11',
 'OS13',
 'OS12',
 'OS14',
 'OS15',
 'OS16',
 'OS17',
 'OS18',
 'OS19',
 'OS20',
 'OS21',
 'OS22',
 'WHO5B1',
 'WHO5B2',
 'WHO5B3',
 'WHO5B4',
 'WHO5B5',
 'HPQB1',
 'HPQB2',
 'HPQB3',
 'HPQB4',
 'HPQB5',
 'HPQB6',
 'HPQB7',
 'HPQB8',
 'WHO5S1',
 'WHO5S2',
 'WHO5S3',
 'WHO5S4',
 'WHO5S5',
 'HPQS1',
 'HPQS2',
 'HPQS3',
 'HPQS4',
 'HPQS5',
 'HPQS6',
 'HPQS7',
 'HPQS8',
 'PerformanceOthers',
 'PerB1item',
 'PerS1item',
 'Temp1',
 'Temp2',
 'Temp3',
 'Temp5',
 'Temp6',
 'Temp7',
 'Temp7.1',
 'Temp9',
 'Temp10',
 'Temp11',
 'Temp24',
 'Temp25',
 'Temp26',
 'Temp27',
 'Temp28',
 'Temp29',
 'Temp30',
 'Temp31',
 'Temp32',
 'Temp33',
 'Temp34',
 'Temp35',
 'Temp36',
 'Temp37',
 'Temp38',
 'Temp39',
 'Temp40',
 'Temp41',
 'Temp42',
 'Temp43',
 'Temp44',
 'Temp45',
 'Temp46',
 'Temp47',
 'Temp48',
 'Temp49',
 'Temp50',
 'Temp51',
 'Temp52',
 'Temp53',
 'Temp54',
 'Temp55',
 'Temp56',
 'Temp57',
 'Temp58',
 'Temp59',
 'Temp60',
 'Temp64',
 'Temp65',
 'Temp65.1',
 'Temp66',
 'Temp67',
 'Temp68',
 'Temp69',
 'Temp70',
 'Temp71',
 'Temp72',
 'Temp73',
 'Temp74',
 'Temp75',
 'Temp76',
 'Temp77',
 'Temp78',
 'Temp79',
 'Temp80',
 'Temp81',
 'Temp82',
 'Temp83',
 'Temp84',
 'Temp85',
 'Temp86',
 'Temp87',
 'Temp88',
 'Temp90',
 'Temp91',
 ])

# merge covid_df V1 and V2

In [383]:
print('before')
print(covid_df.shape)
print(covid_df_v1.shape)

#drop those Temp column
# https://www.skytowner.com/explore/dropping_columns_whose_label_contains_a_substring_in_pandas

covid_df = covid_df.loc[:,~covid_df.columns.str.contains('Temp')]
covid_df_v1 = covid_df_v1.loc[:,~covid_df_v1.columns.str.contains('Temp')]

print('after')
print(covid_df.shape)
print(covid_df_v1.shape)

print('merge')
covid_df = pd.concat([covid_df,covid_df_v1])
print(covid_df.shape)

before
(167, 165)
(88, 165)
after
(167, 90)
(88, 90)
merge
(255, 90)


# start preprocessing

In [384]:
# add id column
# https://stackoverflow.com/questions/38862293/how-to-add-incremental-numbers-to-a-new-column-using-pandas
covid_df.insert(0, 'ID', range(1, 1 + len(covid_df)))

In [385]:
# remove unnamed column
# del covid_df['Unnamed: 0']

In [386]:
# move Timestamp column to the beginning
covid_df.insert(0, 'Timestamp', covid_df.pop('Timestamp'))

In [387]:
# remove Email column
# del covid_df['Email Address']

In [388]:
# remove Confirm_REMOVE column
del covid_df['Confirm_REMOVE']

In [389]:
# filter Office_work_before_REMOVE with no answer
covid_df.shape
covid_df = covid_df[covid_df['Office_work_before_REMOVE'] == 'ใช่']
del covid_df['Office_work_before_REMOVE']
covid_df.shape

(223, 89)

In [391]:
# fill Language with 'Thai'
covid_df['Language'] = 'Thai'

In [390]:
#covid_df
covid_df.to_csv('covid_data.csv', index=False)

## Isolation

In [398]:
covid_df.Isolation.value_counts()

ข้าพเจ้าจะออกจากบ้านเพื่อไปทำกิจกรรมที่สำคัญเท่านั้น (เช่น ซื้ออาหาร พบแพทย์/รับการรักษา ออกกำลังกาย)    145
ข้าพเจ้าออกจากบ้านสม่ำเสมอเพื่อไปทำงานหรือผ่อนคลาย แต่ข้าพเจ้ารักษาระยะห่างกับผู้อื่น                     70
ข้าพเจ้าออกจากบ้านสม่าเสมอ และข้าพเจ้าไม่ได้รักษาระยะห่างกับผู้อื่น                                        4
ข้าพเจ้าจะไม่ออกจากบ้าน ไม่ว่าจะด้วยเหตุผลใดก็ตาม                                                          4
Name: Isolation, dtype: int64